#Stock Recommendation Agent

This code implements an AI-powered stock recommendation system using CrewAI. The system analyzes financial data for a specified company or stock, evaluates its performance and market trends, and provides investment recommendations. It's designed to help investors make informed decisions by leveraging multiple specialized AI agents that work together to gather information, analyze data, and generate actionable insights.

##Setup and Dependencies
This section imports all necessary libraries. CrewAI requires several key components:

- Agent, Task, Crew, and Process are core CrewAI classes
- LLM is used to specify which language model to use
- WebsiteSearchTool and SerperDevTool are tools agents can use to search the web

In [ ]:
!pip install --q crewai crewai-tools python-dotenv youtube_transcript_api pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.4 MB/s eta 0:

In [ ]:
# Environmental variables
import os
from dotenv import load_dotenv
from IPython.display import Markdown


# CrewAI
from crewai import Agent,Task, Crew, Process,LLM
from crewai_tools import  WebsiteSearchTool,SerperDevTool

# OpenAI
from openai import OpenAI

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

In [ ]:
# Set API keys (or load them from a .env file)
SERPER_API_KEY = "< put your key here. Get it from https://serper.dev"
# Using lab open AI key
OPENAI_API_KEY = "sk-proj-SCddVD3ZWsbBhv5IScfoT3BlbkFJAJdyR6CDtI7PNCp7Uv0l"
os.environ["SERPER_API_KEY"] = SERPER_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY # Set the OpenAI API key in the environment

In [ ]:
#Initialize the LLM class
OPENAI_LLM = LLM(
    model="openai/gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"],
    temperature = 0
)

##Define Agents

This class defines three different agents with specific roles in the stock recommendation process:

1. Finance News Agent: Gathers and analyzes financial data about a specific entity using web searches.
Uses WebsiteSearchTool() to access online information
Has contextual knowledge about Indian markets


2. Summarization Agent: Simplifies complex financial information into digestible points

3. Recommendation Agent: Evaluates the summarized information to provide investment recommendations

Per CrewAI documentation, each agent requires:

- A clear role (professional identity)
- A specific goal (what it's trying to achieve)
- A backstory (contextual knowledge and motivation)
- Optional tools (capabilities beyond conversation)
- An llm instance to power its reasoning



In [ ]:
class StockRecommenderAgents():

  def finance_news_agent(self,entityname):
    self.entityname=entityname
    return Agent(
        role= "Senior Financial Advisor",
        goal="To analyze recent market trends, financial news, and economic events about {entityname} solely using online sources, \
        providing recommendations regarding if buying a stock or investing in a specific company is a good idea or not.",
        backstory=("You work at the National Stock Exchange in India and are working to provide the best investment recommendations to your customers"
                  "You need to analyze all available data and recommend profitable stocks and investment to your customers about {entityname} without making any assumptions"
                  "Use resources such at reliable websites to generate relevant insights about {entityname} which will be useful in determining if we should purchase the stock or not"),
        Tools=[
            WebsiteSearchTool()
        ],
        llm=OPENAI_LLM
        )

  def summarization_agent(self):
   return Agent(
    role="Summarization writer",
    goal="To summarize the information gathered by the senior financial advisor into short, easy to understand points",
    backstory=("You work as a summary writer who reads a chunck of text and simplifies it into points."),
    llm=OPENAI_LLM
    )

  def recommendation_agent(self):
   return Agent(
    role="Senior Stock Advisor",
    goal="To understand the summary by the senior financial advisor and guage if {entityname} stocks and companies are worth investing in and recommending it to your clients",
    backstory="You are a senior stock recommender with knowledge about the indian stock markets"
    "You can understand which stocks are good investments in the short run and the long run and can give concise recommendations about it",
    llm=OPENAI_LLM
    )


##Define Tasks

This class defines the tasks that agents will perform. According to CrewAI documentation, tasks need:

- A detailed description explaining what the agent should do
- An agent assignment specifying who will perform the task
- An expected_output format that guides the output structure

Each task corresponds to one of the agents:

1. Stock Market Analysis: For the Finance News Agent to gather and analyze data
2. Summarizer: For the Summarization Agent to condense information
3. Recommender: For the Recommendation Agent to make investment recommendations

In [ ]:
class StockRecommenderTasks():
  def stock_market_analysis(self, agent):
        return Task(
            description=('''
            Task: Use online search tools to gather recent financial news and market data.
            Focus on key economic indicators, top-performing stocks, and relevant economic events.
            Summarize these findings to give an overview of current market conditions.

            '''),
            agent=agent,
            expected_output=(f'''A report summarizing recent financial insights including:\n
            1.**{entityname}**:\n is {entityname} a stock worth purchasing?
            - Summary of findings\n'''
        )

        )
  def summarizer(self,agent):
    return Task(
        description=('''
        Task-use information given and summarize in into short easy to understand points out of which recommendations can be easily derived.
        '''),
        agent=agent,
        expected_output=('''
        Summary
        ''')
    )

  def recommender(self,agent):
    return Task(
      description=(
      '''Task-Use information given and derive which stocks are profitable to invest in and give the specific stock names and sectors which are recommended in points'''
      ),
      agent=agent,
      expected_output=('''
      Recommendation about {entityname}
      ''')
  )

##Crew Definition and Execution

This is where everything comes together. The StockRecommenderCrew class:

- Takes an entity name (company/stock) as input
- Creates instances of all agents and tasks
- Assembles them into a Crew
- Kicks off the execution process with crew.kickoff()

According to CrewAI documentation, a Crew needs:

- A list of agents who will be working together
- A list of tasks to be completed
- Optional parameters like verbose=True for detailed execution logs

In [ ]:
class StockRecommenderCrew():
  def __init__(self, entityname):
    self.entityname = entityname

  def run(self):
    agents = StockRecommenderAgents()
    tasks = StockRecommenderTasks()

    finance_news_agent = agents.finance_news_agent(entityname=self.entityname)
    stock_market_analysis_task = tasks.stock_market_analysis(finance_news_agent)
    summarization_agent = agents.summarization_agent()
    summarization_task = tasks.summarizer(summarization_agent)
    recommendation_agent = agents.recommendation_agent()
    recommendation_task = tasks.recommender(recommendation_agent)

    crew = Crew(
        agents=[finance_news_agent,summarization_agent,recommendation_agent],
        tasks=[stock_market_analysis_task,summarization_task,recommendation_task],
        verbose=True
    )

    result = crew.kickoff()
    return result

This section runs the application when executed directly:

- Prompts the user for an entity name (e.g., "Tata Power")
- Creates a new instance of the StockRecommenderCrew with that entity
- Runs the crew to get the recommendation
- Prints the formatted results

In [ ]:
if __name__ == "__main__":
  print("## Welcome to Stock recommender Crew")
  print('-------------------------------')
  entityname = input()
  StockRecommenderCrew = StockRecommenderCrew(entityname)
  result = StockRecommenderCrew.run()
  print("\n\n########################################")
  print("##     Here is your Stock Recommendation Advice     ##")
  print("############################################\n")
  print(result)

## Welcome to Stock recommender Crew
-------------------------------
Tata Power


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f409fe9b-6117-44b7-ae4d-77af48285e11                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: dacda208-4429-46b9-8d0e-963458f4dab9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: dacda208-4429-46b9-8d0e-963458f4dab9
       Status: Executing Task...
    └── 🤖 Agent: Senior Financial Advisor
            Status: In Progress

# Agent: Senior Financial Advisor
## Task: 
            Task: Use online search tools to gather recent financial news and market data.
            Focus on key economic indicators, top-performing stocks, and relevant economic events.
            Summarize these findings to give an overview of current market conditions.

            


...

🚀 Crew: crew
├── 📋 Task: dacda208-4429-46b9-8d0e-963458f4dab9
│      Assigned to: Senior Financial Advisor
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Financial Advisor
│           Status: ✅ Completed
├── 📋 Task: 7eba8d24-2843-40df-8eaf-adcb07f6f0df
│      Assigned to: Summarization writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Summarization writer
│           Status: ✅ Completed
└── 📋 Task: 5c4db2bb-8a72-40fc-a586-b6f958800a05
       Status: Executing Task...
    └── 🤖 Agent: Senior Stock Advisor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: dacda208-4429-46b9-8d0e-963458f4dab9
│      Assigned to: Senior Financial Advisor
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Financial Advisor
│           Status: ✅ Completed
├── 📋 Task: 7eba8d24-2843-40df-8eaf-adcb07f6f0df
│      Assigned to: Summarization writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Summarization writer
│           Status: ✅ Completed
└── 📋 Task: 5c4db2bb-8a72-40fc-a586-b6f958800a05
       Assigned to: Senior Stock Advisor
       Status: ✅ Completed
    └── 🤖 Agent: Senior Stock Advisor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5c4db2bb-8a72-40fc-a586-b6f958800a05                                                                     │
│  Agent: Senior Stock Advisor                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f409fe9b-6117-44b7-ae4d-77af48285e11                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



########################################
##     Here is your Stock Recommendation Advice     ##
############################################

**Recommendation about Tata Power**

1. **Company Overview:**
   - Tata Power is a major integrated power company in India, part of the Tata Group, involved in electricity generation, transmission, and distribution.

2. **Recent Financial Performance:**
   - The company is experiencing consistent financial growth, focusing on expanding its renewable energy portfolio, which aligns with global sustainable energy trends.
   - Revenue and profit margins are improving due to increased electricity demand and strategic renewable energy investments.

3. **Market Trends:**
   - The Indian power sector is shifting towards renewable energy, supported by government policies. Tata Power's investments in solar and wind energy position it well to benefit from these trends.
   - Efforts to reduce debt and improve operational efficiencies are positively viewed 

In [ ]:
display(Markdown(result.raw))

**Recommendation about Tata Power**

1. **Company Overview:**
   - Tata Power is a major integrated power company in India, part of the Tata Group, involved in electricity generation, transmission, and distribution.

2. **Recent Financial Performance:**
   - The company is experiencing consistent financial growth, focusing on expanding its renewable energy portfolio, which aligns with global sustainable energy trends.
   - Revenue and profit margins are improving due to increased electricity demand and strategic renewable energy investments.

3. **Market Trends:**
   - The Indian power sector is shifting towards renewable energy, supported by government policies. Tata Power's investments in solar and wind energy position it well to benefit from these trends.
   - Efforts to reduce debt and improve operational efficiencies are positively viewed by investors.

4. **Recent Developments:**
   - Tata Power is expanding its renewable energy capacity with new solar projects and international collaborations to enhance technology and efficiency.
   - The company is also developing smart grid solutions and electric vehicle charging infrastructure, expected to be future growth areas.

5. **Economic Indicators:**
   - The Indian economy is recovering post-pandemic, with increased industrial activity and energy consumption, benefiting power companies like Tata Power.
   - Inflation and interest rates are important factors to monitor as they can affect operational costs and project financing.

6. **Analyst Recommendations:**
   - Financial analysts generally have a positive outlook on Tata Power, recommending it as a 'buy' for long-term investors interested in the renewable energy sector due to its strategic focus and strong market position.

**Conclusion:**
Tata Power is a promising investment, especially for those interested in the renewable energy sector, due to its strong financial performance, market trends, and strategic initiatives. However, investors should consider market volatility and economic conditions before investing.